# Create Translations for DAVIS 2016 Contours

In this notebook translations between points of two consecutive contours are saved as numpy arrarys.

## Example

For two consecutive contours from the annotations of the DAVIS 2016 dataset, the closest points are matched and then the translations are calculated and saved as a numpy array. Initially, there are twice as many contour points on the second contour, so that each point from the first contour finds a close neighbour. After the matching, the points from the second contour that are not matched get discarded and the contour is saved again as a numpy array.

In this picture, the points from the first contour are in red and the one from the second contour in green. The small blue arrows represent the translations of each point.

<img src="Images/bear_annotation_translations.jpeg" alt="bear_annotation_translations" width="512"/>

## Imports

In [1]:
import numpy as np
import os
from scipy import spatial

## Paths

In [2]:
CONTOURS_FOLDERS_PATH = 'DAVIS_2016/DAVIS/Contours/480p'
TRANSLATIONS_FOLDERS_PATH = 'DAVIS_2016/DAVIS/Translations/480p'

## Functions

In [3]:
def get_min_indices(distances):
    '''Returns a (N, 2) array containing the indices of the smallest distances
       sorted in ascending order given a (N, N) distance matrix.'''
    
    # Get indices that sort array by minimal distance
    min_indices = np.argsort(distances, axis=None)

    # Turn array of flat indices into a tuple of coordinate arrays
    min_indices = np.unravel_index(min_indices, distances.shape)

    # Turn tuple of arrays into array of coordinate arrays
    min_indices = np.array([min_indices[0], min_indices[1]])

    # Transpose array
    min_indices = min_indices.T

    return min_indices

In [4]:
def get_corresponding_points(contour_0, contour_1):
    '''Returns a (N, 2) array containing the matching of the closest contour 
       points given two contours.'''
    
    # Get distance matrix between every point of contour_0 and contour_1
    distances = spatial.distance.cdist(contour_0, contour_1, 'euclidean')
    
    # Get indices of the smallest distances
    min_indices = get_min_indices(distances)
    
    # Compute corresponding points. Each point can only be matched once
    corresponding_points = np.empty([0, 2], int)
    forbidden_points_0 = set()
    forbidden_points_1 = set()
    
    for min_index in min_indices:
        if ((min_index[0] in forbidden_points_0) or 
            (min_index[1] in forbidden_points_1)):
            continue
        else:
            corresponding_points = np.append(corresponding_points, 
                                             min_index.reshape([1,2]),
                                             axis=0)
            forbidden_points_0.add(min_index[0])
            forbidden_points_1.add(min_index[1])
    
    # Sort corresponding points by first column
    corresponding_points = corresponding_points[corresponding_points[:,0].argsort()]
    
    return corresponding_points

In [5]:
def get_translations(contour_0, contour_1):
    '''Returns the translations for each point in contour_0 to contour_1.'''
    
    # Get corresponding points
    corresponding_points = get_corresponding_points(contour_0, contour_1)
    
    # Compute translations
    translations = np.empty([0, 2], int)
    
    for point in corresponding_points:
        translation = np.subtract(contour_1[point[1]], contour_0[point[0]])
        translations = np.append(translations, translation.reshape([1,2]), axis=0)
    
    return translations, corresponding_points 

In [6]:
def create_translations_for_all_contours(contours_folders_path,
                                         translations_folders_path):
    '''Creates translations for contours and saves them as numpy arrays.'''
    
    # Get list of contours folder (there is one for each sequence)
    contours_folders_list = os.listdir(contours_folders_path)

    # Iterate through folders
    for i, folder in enumerate(contours_folders_list):
        
        # Debug
        #if (i > 1): break

        print('#{}: {}'.format(i, folder))

        # Create folder to save translations
        translations_folder = os.path.join(translations_folders_path, folder)
        if not os.path.exists(translations_folder):
            os.makedirs(translations_folder)

        # Get list of contours (there is one for each frame)
        contours = os.listdir(os.path.join(contours_folders_path, folder))
        if '.ipynb_checkpoints' in contours:
            contours.remove('.ipynb_checkpoints')
        contours.sort()
            
        # Iterate through annotations
        for j, contour in enumerate(contours):

            # Debug
            #if (j > 1): break
                
            # Load contours
            contour_0_path = os.path.join(contours_folders_path, folder, contours[j])
            contour_0 = np.load(contour_0_path)
            
            try:
                contour_1_path = os.path.join(contours_folders_path, folder, contours[j+1])
                contour_1 = np.load(contour_1_path)
            except IndexError as e:
                break

            # Get translations
            translations_0_1, corresponding_points = get_translations(contour_0, contour_1)
            
            # Update contour_1 so that it has same amount of points as contour_0
            corresponding_points = corresponding_points.T 
            contour_1 = contour_1[corresponding_points[1]] 

            # Save contour_1
            np.save(contour_1_path, contour_1)
            
            # Save translations
            np.save(os.path.join(translations_folder, contour[:5]), translations_0_1)

## Create Translations

In [7]:
create_translations_for_all_contours(CONTOURS_FOLDERS_PATH,
                                     TRANSLATIONS_FOLDERS_PATH)

#0: swing
#1: drift-chicane
#2: lucia
#3: soapbox
#4: breakdance
#5: drift-turn
#6: mallard-fly
#7: motorbike
#8: scooter-gray
#9: scooter-black
#10: breakdance-flare
#11: bus
#12: elephant
#13: bmx-trees
#14: rollerblade
#15: dance-twirl
#16: dance-jump
#17: horsejump-high
#18: mallard-water
#19: car-turn
#20: kite-walk
#21: dog-agility
#22: car-shadow
#23: paragliding-launch
#24: stroller
#25: bear
#26: hockey
#27: dog
#28: boat
#29: car-roundabout
#30: soccerball
#31: train
#32: tennis
#33: parkour
#34: surf
#35: kite-surf
#36: cows
#37: drift-straight
#38: flamingo
#39: goat
#40: rhino
#41: hike
#42: motocross-jump
#43: horsejump-low
#44: motocross-bumps
#45: libby
#46: paragliding
#47: camel
#48: blackswan
#49: bmx-bumps
